In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Data Quality").getOrCreate()
spark


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import random

# Start Spark session
spark = SparkSession.builder.appName("FakeTrainingData").getOrCreate()

# Generate fake data
data = [
    (44, 9, 21.0),
    (10, 5, 53.0),
    (37, 6, 80.0),
    (19, 10, 11.0),
    (46, 13, 215.0)
]

# Add more rows to reach 543 samples
for _ in range(543 - len(data)):
    totalAdClicks = random.randint(1, 60)
    totalBuyClicks = random.randint(1, 15)
    totalRevenue = round(random.uniform(5, 250), 1)
    data.append((totalAdClicks, totalBuyClicks, totalRevenue))

# Define schema
columns = ["SumAddClicks", "SumBuyClicks", "Revenue"]




In [6]:
trainingDF = spark.createDataFrame(data, columns)

# Show first 5 rows
trainingDF.show(5)

# Show dimensions (rows, columns)
rows = trainingDF.count()
cols = len(trainingDF.columns)
print((rows, cols))

+------------+------------+-------+
|SumAddClicks|SumBuyClicks|Revenue|
+------------+------------+-------+
|          44|           9|   21.0|
|          10|           5|   53.0|
|          37|           6|   80.0|
|          19|          10|   11.0|
|          46|          13|  215.0|
+------------+------------+-------+
only showing top 5 rows

(543, 3)


In [7]:
rows.shape()

AttributeError: 'int' object has no attribute 'shape'